# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


SparkSQL

In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("UserRoutesSQL") \
    .master("local[*]") \
    .getOrCreate()

# Read the data from HDFS
df = spark.read.csv(
    "hdfs:/data/clickstream.csv",
    header=True,
    inferSchema=True,
    sep='\t',
    quote='"',
    escape='"'
)


df.createOrReplaceTempView('clickstream')

# Execute the SQL query with LIMIT 30
result = spark.sql("""
WITH error_events AS (
    SELECT
        user_id,
        session_id,
        MIN(CASE WHEN LOWER(event_type) LIKE '%error%' THEN timestamp END) AS min_error_timestamp
    FROM clickstream
    GROUP BY user_id, session_id
),
filtered_events AS (
    SELECT
        c.user_id,
        c.session_id,
        c.event_type,
        c.event_page,
        c.timestamp,
        e.min_error_timestamp
    FROM clickstream c
    LEFT JOIN error_events e
        ON c.user_id = e.user_id AND c.session_id = e.session_id
    WHERE e.min_error_timestamp IS NULL OR c.timestamp <= e.min_error_timestamp
),
page_events AS (
    SELECT
        user_id,
        session_id,
        event_page,
        timestamp,
        ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS rn,
        LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS prev_event_page
    FROM filtered_events
    WHERE event_type = 'page'
),
distinct_page_events AS (
    SELECT
        user_id,
        session_id,
        event_page,
        rn
    FROM page_events
    WHERE event_page != prev_event_page OR prev_event_page IS NULL
),
routes AS (
    SELECT
        user_id,
        session_id,
        CONCAT_WS('-', TRANSFORM(
            ARRAY_SORT(COLLECT_LIST(NAMED_STRUCT('rn', rn, 'event_page', event_page))),
            x -> x.event_page)) AS route
    FROM distinct_page_events
    GROUP BY user_id, session_id
),
route_counts AS (
    SELECT
        route,
        COUNT(*) AS count
    FROM routes
    GROUP BY route
)
SELECT
    route,
    count
FROM route_counts
ORDER BY count DESC
LIMIT 30
""")

# Collect the results
top_routes_list = result.collect()

# Save the results to 'result.txt'
with open('result.txt', 'w', encoding='utf-8') as f:
    for row in top_routes_list:
        f.write(f"{row['route']}:{row['count']}\n")


spark.stop()

SparkRDD

In [ ]:
from pyspark import SparkContext


sc = SparkContext(appName="UserRoutesRDD")


lines = sc.textFile("hdfs:/data/clickstream.csv")

# Remove the header
header = lines.first()
lines = lines.filter(lambda x: x != header)

# Parse each line
def parse_line(line):
    import csv
    from io import StringIO

    # Use csv reader to handle quoted fields and separators
    reader = csv.reader(StringIO(line), delimiter='\t', quotechar='"')
    fields = next(reader)

    # Parse fields
    user_id = int(fields[0])
    session_id = int(fields[1])
    event_type = fields[2]
    event_page = fields[3]
    timestamp = int(fields[4])

    return (user_id, session_id, event_type, event_page, timestamp)

parsed_lines = lines.map(parse_line)

# Key by (user_id, session_id)
session_events = parsed_lines.map(lambda x: ((x[0], x[1]), x))

# Group events by session
session_grouped = session_events.groupByKey()

# Process each session
def process_session(events):
    # Convert to list for multiple iterations
    events_list = list(events)

    # Sort events by timestamp
    events_sorted = sorted(events_list, key=lambda x: x[4])

    # Find the earliest error timestamp
    min_error_timestamp = None
    for event in events_sorted:
        event_type = event[2]
        timestamp = event[4]
        if 'error' in event_type.lower():
            min_error_timestamp = timestamp
            break  # Earliest error timestamp found

    # Process events up to and including the error timestamp
    route_pages = []
    last_page = None
    for event in events_sorted:
        event_type = event[2]
        event_page = event[3]
        timestamp = event[4]

        if min_error_timestamp is not None and timestamp > min_error_timestamp:
            break  # Stop processing events after error timestamp

        if event_type == 'page':
            # Remove consecutive duplicates
            if event_page != last_page:
                route_pages.append(event_page)
                last_page = event_page

    if route_pages:
        route = '-'.join(route_pages)
        return route
    else:
        return None

routes = session_grouped.mapValues(process_session)

# Filter out sessions with empty or None routes
routes_filtered = routes.filter(lambda x: x[1])

# Count route frequencies
route_counts = routes_filtered.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

# Get top 30 routes
route_counts_swapped = route_counts.map(lambda x: (x[1], x[0]))
top_30 = route_counts_swapped.top(30)
top_30_routes = sc.parallelize(top_30).map(lambda x: (x[1], x[0]))

# Collect results
top_routes_list = top_30_routes.collect()


with open('result.txt', 'w', encoding='utf-8') as f:
    for route, count in top_routes_list:
        f.write(f"{route}:{count}\n")


sc.stop()

SparkDF

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, lower, lag, collect_list, concat_ws, min as spark_min
)
from pyspark.sql.window import Window


spark = SparkSession.builder \
    .appName("UserRoutesDF") \
    .master("local[*]") \
    .getOrCreate()


df = spark.read.csv(
    "hdfs:/data/clickstream.csv",
    header=True,
    inferSchema=True,
    sep='\t',
    quote='"',
    escape='"'
)

# Identify error events
df = df.withColumn('is_error', when(lower(col('event_type')).like('%error%'), 1).otherwise(0))

# Get earliest error timestamp per session
window_session = Window.partitionBy('user_id', 'session_id')

df = df.withColumn('error_timestamp', when(col('is_error') == 1, col('timestamp')))
df = df.withColumn('min_error_timestamp', spark_min('error_timestamp').over(window_session))

# Include events up to and including the error timestamp
df_filtered = df.filter(
    (col('min_error_timestamp').isNull()) | (col('timestamp') <= col('min_error_timestamp'))
)

# Select only 'page' events
df_pages = df_filtered.filter(col('event_type') == 'page')

# Remove consecutive duplicates within sessions
window_ordered = Window.partitionBy('user_id', 'session_id').orderBy('timestamp')
df_pages = df_pages.withColumn('prev_event_page', lag('event_page').over(window_ordered))

df_pages_distinct = df_pages.filter(
    (col('event_page') != col('prev_event_page')) | col('prev_event_page').isNull()
)

# Collect routes for each session
df_routes = df_pages_distinct.groupBy('user_id', 'session_id').agg(
    collect_list('event_page').alias('pages_list')
)

# Create route strings
df_routes = df_routes.withColumn('route', concat_ws('-', 'pages_list'))

# Count route frequencies
df_route_counts = df_routes.groupBy('route').count()

# Get top 30 routes
top_routes = df_route_counts.orderBy(col('count').desc()).limit(30)


top_routes_list = top_routes.collect()

# Save the results to 'result.txt'
with open('result.txt', 'w', encoding='utf-8') as f:
    for row in top_routes_list:
        f.write(f"{row['route']}:{row['count']}\n")


spark.stop()
